In [5]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [ ]:
main_dir = ""  # Set your main directory here

# Parameters
img_size = (256, 256)
data_train = []
labels_train = []
data_test = []
labels_test = []

# Load training data
train_dir = os.path.join(main_dir, "train")
for folder in os.listdir(train_dir):
    folder_path = os.path.join(train_dir, folder)
    if os.path.isdir(folder_path):
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, img_size)
            data_train.append(img)
            labels_train.append(folder)

# Load test data
test_dir = os.path.join(main_dir, "test")
for folder in os.listdir(test_dir):
    folder_path = os.path.join(test_dir, folder)
    if os.path.isdir(folder_path):
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, img_size)
            data_test.append(img)
            labels_test.append(folder)

# Convert to numpy arrays
data_train = np.array(data_train)
labels_train = np.array(labels_train)
data_test = np.array(data_test)
labels_test = np.array(labels_test)

# Normalize
data_train = data_train.astype('float32') / 255.0
data_test = data_test.astype('float32') / 255.0

# Reshape for CNN
X_train = data_train.reshape(-1, img_size[0], img_size[1], 1)
X_test = data_test.reshape(-1, img_size[0], img_size[1], 1)

# Shuffle training data
train_indices = np.arange(X_train.shape[0])
np.random.shuffle(train_indices)
X_train = X_train[train_indices]
labels_train = labels_train[train_indices]

# Shuffle test data (optional)
test_indices = np.arange(X_test.shape[0])
np.random.shuffle(test_indices)
X_test = X_test[test_indices]
labels_test = labels_test[test_indices]

# Display first five training imag
plt.figure(figsize=(10, 5))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(X_train[i].reshape(img_size[0], img_size[1]), cmap='gray')
    plt.title(labels_train[i])
    plt.axis('off')
plt.show()

# Display first five test imag
plt.figure(figsize=(10, 5))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(X_test[i].reshape(img_size[0], img_size[1]), cmap='gray')
    plt.title(labels_test[i])
    plt.axis('off')
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

main_dir = ""  

# Parameters
img_size = (256, 256)
batch_size = 32

# Create ImageDataGenerator instances for training and testing
datagen_train = ImageDataGenerator(rescale=1./255, 
                                    validation_split=0.4)
datagen_test = ImageDataGenerator(rescale=1./255)

# Prepare training dataset
train_generator = datagen_train.flow_from_directory(
    os.path.join(main_dir, "train"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  
    subset='training'  
)

# Prepare validation dataset
validation_generator = datagen_train.flow_from_directory(
    os.path.join(main_dir, "train"),  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical', 
    subset='validation'  
)

# Prepare test dataset
test_generator = datagen_test.flow_from_directory(
    os.path.join(main_dir, "test"), 
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  
)

# Get class names
classnames = list(train_generator.class_indices.keys())
print('Data generators ready')


In [ ]:
def display_images(generator, title):
    plt.figure(figsize=(10, 10))
    for i in range(5):
        img, label = next(generator)
        plt.subplot(1, 5, i + 1)
        plt.imshow(img[0])  
        plt.title(classnames[label.argmax()])  
        plt.axis('off')
    plt.suptitle(title)
    plt.show()


display_images(train_generator, "First 5 Training Images")

display_images(validation_generator, "First 5 Validation Images")

display_images(test_generator, "First 5 Test Images")

In [8]:
import tensorflow
from  tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
from keras.regularizers import l2

# Define the regularization factor (this can be tuned)
l2_reg = 1e-2  # Example value, you can adjust this

model = Sequential()

input_shape = train_generator.image_shape

model.add(Conv2D(64, (10, 10), input_shape=input_shape, activation='relu', kernel_regularizer=l2(l2_reg)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (10, 10), activation='relu', kernel_regularizer=l2(l2_reg)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (10, 10), activation='relu', kernel_regularizer=l2(l2_reg)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (10,10), activation='relu', kernel_regularizer=l2(l2_reg)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu', kernel_regularizer=l2(l2_reg)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(512, activation='relu', kernel_regularizer=l2(l2_reg)))  
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu', kernel_regularizer=l2(l2_reg)))

# Output layer
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Compile the model with AdamW optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.AdamW(learning_rate=0.00005, weight_decay=1e-5),
              metrics=['accuracy'])

# Print the summary of the model
print(model.summary())

In [ ]:
steps_per_epoch = train_generator.samples // batch_size
validation_steps = validation_generator.samples // batch_size

# Define the filepath for saving the best model
checkpoint_filepath = 'best_sign_model_de.keras'

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,  # Only save the best model
    monitor='val_loss',   # Monitor validation loss
    mode='min',           # Save the model with the minimum validation loss
    verbose=1
)

#model=load_model("best_sign_model.keras")
# Train the model and save the best model based on validation loss
history = model.fit(train_generator,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=validation_generator,
                    validation_steps=validation_steps,
                    epochs=100,
                    callbacks=[checkpoint_callback])  # Include the checkpoint callback

# Save the entire model after training (optional)
model.save("sign_det.keras")  # Saving as a .keras file

# Load the best model after training
best_model = load_model("best_sign_model_de.keras")

# Print the summary of the best model
print(best_model.summary())

In [ ]:
best_model = load_model("best_sign_model.keras")
test_loss, test_accuracy = best_model.evaluate(test_generator, steps=test_generator.samples // batch_size)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

def evaluate_model(model, test_generator):
    # Ensure steps cover all samples
    steps = int(np.ceil(test_generator.samples / test_generator.batch_size))

    # Predict using the test generator
    y_pred = model.predict(test_generator, steps=steps)

    # Convert predictions to class labels (argmax for multi-class classification)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Get the true class labels from the test generator
    y_true = test_generator.classes

    # Ensure that y_pred_classes and y_true are the same length
    if len(y_pred_classes) != len(y_true):
        print(f"Warning: Mismatch in the number of predictions ({len(y_pred_classes)}) and true labels ({len(y_true)}). Truncating predictions.")
        y_pred_classes = y_pred_classes[:len(y_true)]  # Truncate predictions if necessary

    # Create the confusion matrix
    cm = confusion_matrix(y_true, y_pred_classes)

    # Print the confusion matrix for reference
    print("Confusion Matrix (Numeric):")
    print(cm)

    # Initialize lists to store TP, TN, FP, FN for each class
    summary_data = []

    total_tp = 0
    total_fp = 0
    total_fn = 0

    for i in range(len(cm)):
        TP = cm[i, i]
        FP = cm[:, i].sum() - TP
        FN = cm[i, :].sum() - TP
        TN = cm.sum() - (FP + FN + TP)

        # Calculate Precision and Recall
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0

        # Append results to the summary data list
        summary_data.append({
            'Class': i, 
            'TP': TP, 
            'TN': TN, 
            'FP': FP, 
            'FN': FN,
            'Precision': precision,
            'Recall': recall
        })

        # Accumulate totals for overall precision and recall
        total_tp += TP
        total_fp += FP
        total_fn += FN

    # Convert summary data to DataFrame
    summary_df = pd.DataFrame(summary_data)

    # Calculate overall Precision and Recall
    overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0

    # Display the summary table
    print("\nSummarized Confusion Matrix:")
    print(summary_df)

    # Print overall Precision and Recall
    print(f"\nOverall Precision: {overall_precision:.4f}")
    print(f"Overall Recall: {overall_recall:.4f}")

# Call the evaluate_model function with the best_model and test_generator
evaluate_model(best_model, test_generator)


In [ ]:
test_generator.classes

In [ ]:
def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.show()

# Plot the training history using the history returned from model.fit()
plot_training_history(history)
    